In [5]:
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader 
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
train  = datasets.MNIST(root="data",download=True,train=True,transform=ToTensor())

In [7]:
dataset = DataLoader(train,32)
#image shape (1,28,28)

In [8]:
import torch
from torch.autograd import Variable 
dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor #to run on gpu

N, D_in , H, D_out = 64, 1000,100,10
x = Variable(torch.randn(N,D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N,D_out).type(dtype), requires_grad=False)

w1 = Variable(torch.randn(D_in,H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H,D_out).type(dtype), requires_grad=True)

In [9]:
learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    ##mm is matrix multiplication and clamp , clamps both all the values in the input range between a min and max values

    #calculate loss
    '''Now to compute the loss we are using the same popular loss function , sum of all squared difference of y_pred and y 
Loss is a variable of shape(1,) and loss.data is a Tensor of shape (1,) ; loss.data[0] is a scalar value holding loss'''
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data)
    '''Now we need to set the gradients of both sets of weights to 0 since graident buffers needs to be reset before fresh gradients are calculated'''

    '''now we will use autograd to compute the backward pass . this call will compute the gradient of loss wrt all variables with ```requires_grad=True``` 
after this w1.grad and w2.grad will be variables holding the grad of the loss wrt w1 and w2'''
    loss.backward()

    '''now we will update the weights using gradient descent . for this we will just mutate the values of w1 and w2 in place .'''
    w1.data -= learning_rate*w1.grad.data
    w2.data -= learning_rate*w2.grad.data
    w1.grad.data.zero_()
    w2.grad.data.zero_()

0 tensor(34517408., device='cuda:0')
1 tensor(32761740., device='cuda:0')
2 tensor(33295224., device='cuda:0')
3 tensor(30725706., device='cuda:0')
4 tensor(23829932., device='cuda:0')
5 tensor(15133253., device='cuda:0')
6 tensor(8437664., device='cuda:0')
7 tensor(4563393., device='cuda:0')
8 tensor(2665402., device='cuda:0')
9 tensor(1751278.2500, device='cuda:0')
10 tensor(1282263., device='cuda:0')
11 tensor(1009804.6250, device='cuda:0')
12 tensor(830424.8125, device='cuda:0')
13 tensor(699954.1250, device='cuda:0')
14 tensor(598413.5625, device='cuda:0')
15 tensor(516436.3125, device='cuda:0')
16 tensor(448631.0625, device='cuda:0')
17 tensor(391791.4062, device='cuda:0')
18 tensor(343593.0625, device='cuda:0')
19 tensor(302521.2188, device='cuda:0')
20 tensor(267281.0312, device='cuda:0')
21 tensor(236884.1562, device='cuda:0')
22 tensor(210558.5469, device='cuda:0')
23 tensor(187632.3438, device='cuda:0')
24 tensor(167633.6875, device='cuda:0')
25 tensor(150137.5156, device='c

In [10]:
import math
from torch import nn
from torch import save,load
from torch.nn import functional as F
from torch.optim import Adam
#trying using MyReLU
class MyReLU(nn.Module):
    def forward(self, input):
        self.save_for_backward(input)
        return input.clamp(min=0)
    def backward(self, grad_output):
        input, = self.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input<0] = 0
        return grad_input
    def save_for_backward(self,x):
        self.saved_tensors = x
class MyConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels // groups, *kernel_size))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return F.conv2d(input, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


In [11]:
class ImageClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            MyConv2d(1, 32, (3, 3)),
            MyReLU(),
            MyConv2d(32, 64, (3, 3)),
            MyReLU(),
            MyConv2d(64, 64, (3, 3)),
            MyReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10))
            
    def forward(self, x):
        return self.model(x)


In [12]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.optim import Adam
from tqdm import tqdm
#eval method from pytorch

train_loader = DataLoader(train, batch_size=64, shuffle=True)

def evaluate(model, data_loader, loss_fn):
    model.eval()  # Set the model to evaluation mode

    # Initialize the total loss and correct predictions
    total_loss = 0
    num_correct = 0

    # Loop over the validation data
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in data_loader:
            # Move the input and label tensors to the GPU
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Update the total loss and number of correct predictions
            total_loss += loss.item() * inputs.size(0)
            num_correct += (outputs.argmax(dim=1) == labels).sum().item()

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = num_correct / len(data_loader.dataset)

    return avg_loss, accuracy


if __name__ == '__main__':
    model = ImageClassification().to('cuda')

    # Define the optimizer and loss function
    opt = Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    # Load the training and validation data
    train = MNIST(root='data', train=True, transform=ToTensor(), download=True)
    val_loader = DataLoader(MNIST(root='data', train=False, transform=ToTensor()), batch_size=64)
    # Loop over the number of epochs
    for epoch in range(10):
        # Loop over the training data
        for inputs, labels in tqdm(train_loader):
            # Move the input and label tensors to the GPU
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            # Zero out the gradients of the optimizer
            opt.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Backward pass
            loss.backward()

            # Update the model weights
            opt.step()

        # Evaluate the model on the validation set
        val_loss, val_acc = evaluate(model, val_loader, loss_fn)

        # Print the epoch number, validation loss, and validation accuracy
        print(f'Epoch {epoch+1}: val_loss = {val_loss:.4f}, val_acc = {val_acc:.4f}')
    

100%|██████████| 938/938 [00:08<00:00, 107.61it/s]


Epoch 1: val_loss = 0.0516, val_acc = 0.9827


100%|██████████| 938/938 [00:08<00:00, 113.34it/s]


Epoch 2: val_loss = 0.0366, val_acc = 0.9880


100%|██████████| 938/938 [00:08<00:00, 112.59it/s]


...

Epoch 8: val_loss = 0.0575, val_acc = 0.9884


100%|██████████| 938/938 [00:08<00:00, 110.39it/s]


Epoch 9: val_loss = 0.0566, val_acc = 0.9872


100%|██████████| 938/938 [00:08<00:00, 112.14it/s]


Epoch 10: val_loss = 0.0569, val_acc = 0.9876


In [15]:
# Save the model
torch.save(model.state_dict(), 'model.pt')


In [14]:
from PIL import Image
from torchvision.transforms import ToTensor
import torch

# Preprocess the image
image = Image.open('/home/kalyan/gitrepo/AI-ML-Playground/PyTorch/MNIST/data/sample_image.png')  # Load the image
image = image.resize((28, 28))  # Resize the image
image = ToTensor()(image)  # Convert the image to a tensor
image = image.unsqueeze(0)  # Add a batch dimension
image = image.to('cuda')  # Move the image to the GPU

# Forward pass
output = model(image)

# Get the predicted class
_, pred = output.max(dim=1)
pred = pred.item()

print(f'Predicted class: {pred}')


Predicted class: 7
